In [ ]:
import torch
import pywt
import pandas               as  pd
import numpy                as  np
import matplotlib.pyplot    as  plt
import torch.nn             as  nn

# Hyper Parameters

In [ ]:
# Dataset
samples_per_test    = 32768
Total_test          = 29
test_Duration       = 2

# model         Maybe adding dropout in LSTM is good
input_horizon       = 200
# model-LSTM
LSTM_outFeature     = 100
LSTM_NumLayer       = 2
LSTM_hidden_size    = 64

# CWT
Scales              = 50
wavelet_lis         = ['mexh','morl','gaus1','cmor1-1.0']   # Define the wavelet and scales
wavelet             = wavelet_lis[1]                        # You can choose other wavelet functions as well
Coefficient_Real    = True



# Reading dataset
Read the CSV file into a DataFrame

In [ ]:
df = pd.read_csv('Dataset_BinaryClass.csv')
df['Faulty'] = df['Faulty'].replace({False: 0, True: 1})

"""
print(df.head())

its better to keep numpy format because CWT uses numpy and before network make it pytorch

X = torch.tensor(df['Datas' ].values).reshape(Total_test,samples_per_test)
Y = torch.tensor(df['Faulty'].values).reshape(Total_test,samples_per_test)
"""

X_numpy = df['Datas' ].values.reshape(Total_test,samples_per_test)
Y_numpy = df['Faulty'].values.reshape(Total_test,samples_per_test)

# Continuous Wavelet transform (CWT)

In [ ]:
scales = np.arange(1, Scales)

for wavelet in reversed(wavelet_lis):
    # Perform Continuous Wavelet Transform
    print(wavelet)
    coefficients, frequencies = pywt.cwt(X_numpy[1,:input_horizon], scales, wavelet)

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.imshow(np.abs(coefficients), extent=[0, 1, 1, Scales], aspect='auto', cmap='jet')
    plt.colorbar(label='Magnitude')
    plt.title('Continuous Wavelet Transform')
    plt.xlabel('Time')
    plt.ylabel('Scale')
    plt.show()

In [ ]:
if Coefficient_Real:
    coefficients = np.abs(coefficients)

In [ ]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self,
                input_size  :int ,
                hidden_size :int    = LSTM_hidden_size,
                num_layers  :int    = LSTM_NumLayer,
                output_size :int    = LSTM_outFeature,
                DropOut     :float  = 0)->None:
        super(LSTMModel, self).__init__()
        self.hidden_size    = hidden_size
        self.num_layers     = num_layers
        
        self.lstm           = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc             = nn.Linear(hidden_size, output_size)
        
    def forward(self, x)->tuple:
        out, (h_state,C_state) = self.lstm(x)
        
        # Decode the hidden state of the last time step using the fully connected layer
        out = self.fc(out[:, -1, :])
        return out,h_state,C_state

in ``` h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) ``` x is input and data will be generated in device

In [ ]:
# Define the input data shape
sequence_length = 49
input_size = 200
output_size = 1  # For example, predicting a single value

# Hyperparameters
hidden_size = 64
num_layers = 2

# Create the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Generate random input data
input_data = torch.randn(1, sequence_length, input_size)  # Batch size of 1 for this example

# Forward pass
output,h_state,C_state = model(input_data)

print("Output shape:", output.shape,h_state.shape,C_state.shape)
